DATA CLEANING CASE STUDY

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno as msno
import datetime as dt

In [2]:
# Read in the dataset
airbnb = pd.read_csv('https://raw.githubusercontent.com/nethajinirmal13/Training-datasets/main/airbnb.csv?raw=true', index_col = 'Unnamed: 0')

In [ ]:
airbnb

In [ ]:
# Above link inside Data set type check
airbnb.info()

In [ ]:
# Check Unique In Every Column
for col in airbnb.columns[0:]:
   print(col, ': ', len(airbnb[col].unique()), 'labels')

In [ ]:
airbnb.isnull().sum()

In [ ]:
# Remove "(" and ")" from coordinates
airbnb['coordinates'] = airbnb['coordinates'].str.replace("(","")
airbnb['coordinates'] = airbnb['coordinates'].str.replace(")","")
# Print the header of the column
airbnb['coordinates'].head()

In [ ]:
# This is above code another Method Strip Function Remove leading and trailing whitespace from the 'coordinates' column
airbnb['coordinates'] = airbnb['coordinates'].str.strip()

# Print the header of the column
print(airbnb['coordinates'].head())


In [ ]:
# Split column into two
lat_long = airbnb['coordinates'].str.split(",", expand = True)
lat_long.head()    # Here Head Means Top Data Incase we need top 10 Data means mention head function inside

In [ ]:
# Assign correct columns to latitude and longitude columns in airbnb
airbnb['latitude'] = lat_long[0]
airbnb['longitude'] = lat_long[1]
# Print the header and confirm new column creation
airbnb.head()

In [ ]:
# Convert latitude and longitude to float
airbnb['latitude'] = airbnb['latitude'].astype('float')
airbnb['longitude'] = airbnb['longitude'].astype('float')
# Print dtypes again
airbnb.dtypes

In [19]:
# Drop coordinates column
airbnb.drop('coordinates', axis = 1, inplace = True) # axis Means axis = 0 Row axis = 1 Column

In [ ]:
# Price before Dollar Symbol Remove
# Remove $ from price before conversion to float
airbnb['price'] = airbnb['price'].str.strip("$")
# Print header to make sure change was done
airbnb['price'].head()

In [ ]:
# Convert price to float
airbnb['price'] = airbnb['price'].astype('float')
# Calculate mean of price after conversion
airbnb['price'].mean()
#airbnb['price'].max()


In [ ]:
# Print header of two columns
airbnb[['listing_added', 'last_review']].head()

In [23]:
# Convert both columns to datetime
airbnb['listing_added'] = pd.to_datetime(airbnb['listing_added'], format = '%Y-%m-%d')
airbnb['last_review'] = pd.to_datetime(airbnb['last_review'], format = '%Y-%m-%d')

In [ ]:
# Print header and datatypes of both columns again
print(airbnb[['listing_added', 'last_review']].head())
print(airbnb[['listing_added', 'last_review']].dtypes)

In [ ]:
airbnb['room_type'].unique()

In [ ]:
# Deal with capitalized values
airbnb['room_type'] = airbnb['room_type'].str.lower()
airbnb['room_type'].unique()

In [ ]:
# Deal with capitalized values
airbnb['room_type'] = airbnb['room_type'].str.lower()
airbnb['room_type'].unique()

In [ ]:
# Deal with trailing spaces
airbnb['room_type'] = airbnb['room_type'].str.strip()
airbnb['room_type'].unique()

In [ ]:
# Replace values to 'Shared room', 'Entire place', 'Private room' and 'Hotel room' (if applicable).
mappings = {'private room': 'Private Room',
           'private': 'Private Room',
           'entire home/apt': 'Entire place',
           'shared room': 'Shared room',
           'home': 'Entire place'}

# Replace values and collapse data
airbnb['room_type'] = airbnb['room_type'].replace(mappings)
airbnb['room_type'].unique()

In [ ]:
# Isolate rows of rating > 5.0
airbnb[airbnb['rating'] > 5.0]

In [31]:
# Drop these rows and make sure we have effected changes
airbnb.drop(airbnb[airbnb['rating'] > 5.0].index, inplace = True)

In [ ]:
%pip install missingno

In [ ]:
# Visualize the missingness
msno.matrix(airbnb)
plt.show()

In [ ]:
# Visualize the missingness on sorted values
msno.matrix(airbnb.sort_values(by = 'rating'))
plt.show()

In [35]:
# Impute missing data
airbnb = airbnb.fillna({'reviews_per_month':0,
                       'number_of_stays':0,
                       '5_stars':0})

# Create is_rated column
is_rated = np.where(airbnb['rating'].isna() == True, 0, 1)
airbnb['is_rated'] = is_rated

In [ ]:
airbnb.isna().sum()

In [ ]:
print(airbnb['price'].dtype)

In [42]:
airbnb.loc[(airbnb['price'].isna()) & (airbnb['room_type'] == 'Private Room'), 'price']

240     NaN
280     NaN
333     NaN
408     NaN
478     NaN
         ..
9655    NaN
9744    NaN
9775    NaN
10009   NaN
10010   NaN
Name: price, Length: 131, dtype: float64

In [ ]:
# Get mean price per room_type
airbnb.groupby('room_type').mean()['price']

In [44]:
# Impute price based on conditions
airbnb.loc[(airbnb['price'].isna()) & (airbnb['room_type'] == 'Entire place'), 'price'] = 209.7
airbnb.loc[(airbnb['price'].isna()) & (airbnb['room_type'] == 'Private Room'), 'price'] = 88.02
airbnb.loc[(airbnb['price'].isna()) & (airbnb['room_type'] == 'Shared room'), 'price'] = 70.41

In [46]:
# Doing some sanity checks on date data
today = dt.date.today()
today

datetime.date(2024, 12, 16)

In [47]:
# Are there reviews in the future?
airbnb[airbnb['last_review'].dt.date > today]

,listing_id,name,host_id,host_name,neighbourhood_full,room_type,price,number_of_reviews,last_review,reviews_per_month,availability_365,rating,number_of_stays,5_stars,listing_added,latitude,longitude,is_rated


In [48]:
# Are there listings in the future?
airbnb[airbnb['listing_added'].dt.date > today]

,listing_id,name,host_id,host_name,neighbourhood_full,room_type,price,number_of_reviews,last_review,reviews_per_month,availability_365,rating,number_of_stays,5_stars,listing_added,latitude,longitude,is_rated


In [49]:
# Are there any listings with listing_added > last_review  Open Hotel before any reviews?
inconsistent_dates = airbnb[airbnb['listing_added'].dt.date > airbnb['last_review'].dt.date]
inconsistent_dates

,listing_id,name,host_id,host_name,neighbourhood_full,room_type,price,number_of_reviews,last_review,reviews_per_month,availability_365,rating,number_of_stays,5_stars,listing_added,latitude,longitude,is_rated
4,22986519,Bedroom on the lively Lower East Side,154262349,Brooke,"Manhattan, Lower East Side",Private Room,160.0,23,2019-06-12,2.29,102,3.822591,27.6,0.649383,2020-10-23,40.71884,-73.98354,1
50,20783900,Marvelous Manhattan Marble Hill Private Suites,148960265,Randy,"Manhattan, Marble Hill",Private Room,93.0,7,2018-10-06,0.32,0,4.868036,8.4,0.609263,2020-02-17,40.87618,-73.91266,1
60,1908852,Oversized Studio By Columbus Circle,684629,Alana,"Manhattan, Upper West Side",Entire place,189.0,7,2016-05-06,0.13,0,4.841204,8.4,0.725995,2017-09-17,40.77060,-73.98919,1
124,28659894,Private bedroom in prime Bushwick! Near Trains!!!,216235179,Nina,"Brooklyn, Bushwick",Private Room,55.0,4,2019-04-12,0.58,358,4.916252,4.8,0.703117,2020-08-23,40.69988,-73.92072,1
511,33619855,Modern & Spacious in trendy Crown Heights,253354074,Yehudis,"Brooklyn, Crown Heights",entire place,150.0,6,2019-05-27,2.50,148,3.462432,7.2,0.610929,2020-10-07,40.66387,-73.93840,1
521,25317793,Awesome Cozy Room in The Heart of Sunnyside!,136406167,Kara,"Queens, Sunnyside",Private Room,65.0,22,2019-06-11,1.63,131,4.442485,26.4,0.722388,2020-10-22,40.74090,-73.92696,1
